In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
from collections import Counter, defaultdict

# List of investors to track
investors = [
    "Bill Ackman", "Benjamin Graham", "Warren Buffett", "John Bogle", "Cathie Wood", 
    "Peter Lynch", "Carl Icahn", "Chamath Palihapitiya", "George Soros", "Sallie Krawcheck",
    "John Templeton", "David Gardner", "Tom Gardner", "Rep. Nancy Pelosi", 
    "Rep. Dan Crenshaw", "Sen. Susan Collins", "Rep. Brian Higgins"
]




In [ ]:
import requests
from bs4 import BeautifulSoup

# CIKs for selected investors or firms
investors = {
    "Warren Buffett": "0001067983",
    "Bill Ackman": "0001336528",
    "Carl Icahn": "0000935214",
    "George Soros": "0001029160",
}

SEC_BASE_URL = "https://www.sec.gov"

# Function to fetch the latest 13F-HR filing link for a given CIK
def get_latest_13f_filing(cik):
    search_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=13F-HR&count=10"
    response = requests.get(search_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        filing_links = [
            SEC_BASE_URL + link["href"]
            for link in soup.find_all("a", href=True)
            if "Archives" in link["href"]
        ]
        return filing_links[0] if filing_links else None
    return None


# Function to extract holdings from a filing
def extract_holdings(filing_url):
    response = requests.get(filing_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table_rows = soup.find_all("tr")
        holdings = []
        for row in table_rows:
            cells = row.find_all("td")
            if len(cells) > 1:
                try:
                    stock = cells[0].text.strip()  # Adjust based on column order
                    shares = cells[1].text.strip().replace(",", "")
                    holdings.append((stock, int(shares)))
                except ValueError:
                    continue
        return holdings
    return []

# Process each investor
for investor, cik in investors.items():
    print(f"\nProcessing {investor}...")
    filing_url = get_latest_13f_filing(cik)
    if filing_url:
        print(f"Filing URL: {filing_url}")
        holdings = extract_holdings(filing_url)
        print(f"Holdings for {investor}:")
        for stock, shares in holdings[:10]:  # Show top 10 holdings
            print(f"  {stock}: {shares} shares")
    else:
        print(f"No filings found for {investor}.")



Processing Warren Buffett...
No filings found for Warren Buffett.

Processing Bill Ackman...
No filings found for Bill Ackman.

Processing Carl Icahn...
No filings found for Carl Icahn.

Processing George Soros...
No filings found for George Soros.
